In [1]:
from SymbolicDirectStiffness import *

The intention of this module is to make solving structures using the direct stiffness method easy and to give users a lot of control over the output and how it is handled. Of course, that means that the program doesn't do much for you besides solving structure. Processing the displacments, reactions is on the user. 

Additonally we can achieve symbolic solutions 

This is the first structure that we will solve for the reactions, kinematic unknowns and displacements for:

![Det Cant](img/SideSway.png)

First define the beam characteristics.

In [57]:
L = 120 / 12 ## feet
P = 5 ## kip
E = 30000 ## kip per square inch
I = 400 ## inches^4
A = 1000000 ## Set high to sim axial rigidity

Next define the structure including it's geometry and loads.

In [58]:
struct = Structure()

struct.add_node('1', (0,0), restraint='fixed')
struct.add_node('2', (0, L))
struct.add_node('3', (L, L))
struct.add_node('4', (L, 0), restraint='fixed')
struct.add_beam('1', ('1', '2'), E=E, A=A, I=I)
struct.add_beam('2', ('2', '3'), E=E, A=A, I=2*I)
struct.add_beam('3', ('3', '4'), E=E, A=A, I=I)

struct.add_load('2', (P,0, 0))
struct.solve()

We can observe the displacements

In [59]:
### displacements
for name, node in struct.nodes.items():
    print(f'Delta_{name}x:\t', node.dx)
    print(f'Delta_{name}y:\t', node.dy)
    print(f'theta_{name}:\t', node.dt)


Delta_1x:	 0
Delta_1y:	 0
theta_1:	 0
Delta_2x:	 2.13686480766229e-5
Delta_2y:	 7.69219408451814e-10
theta_2:	 -8.01455310550253e-7
Delta_3x:	 2.13678147557893e-5
Delta_3y:	 -7.69219408451814e-10
theta_3:	 -8.01392811487739e-7
Delta_4x:	 0
Delta_4y:	 0
theta_4:	 0


Next, we can get the reactions at each node

In [60]:
### Reactions
for name, node in struct.nodes.items():
    print(f'F_{name}x:\t', node.fx)
    print(f'F_{name}y:\t', node.fy)
    print(f'M_{name}:\t', node.moment)

F_1x:	 -2.50003749943751
F_1y:	 -2.30765822535544
M_1:	 13.4619338698478
F_2x:	 5
F_2y:	 0
M_2:	 0
F_3x:	 0
F_3y:	 0
M_3:	 0
F_4x:	 -2.49996250056249
F_4y:	 2.30765822535544
M_4:	 13.4614838765977


Next, we can get the internal forces of each beam

In [49]:
for name, beam in struct.beams.items():
    beg = beam.nodeA.get_name()
    end = beam.nodeB.get_name()
    print(f'N_{beg}{end}:\t', beam.internal[0])
    print(f'V_{beg}{end}:\t', beam.internal[1])
    print(f'M_{beg}{end}:\t', beam.internal[2])
    print(f'N_{end}{beg}:\t', beam.internal[3])
    print(f'V_{end}{beg}:\t', beam.internal[4])
    print(f'M_{end}{beg}:\t', beam.internal[5])
        

N_12:	 -2.14282775550507
V_12:	 2.50002999964000
M_12:	 14.2860612200747
N_21:	 2.14282775550507
V_21:	 -2.50002999964000
M_21:	 10.7142387763254
N_23:	 2.49997000035364
V_23:	 -2.14282775550507
M_23:	 -10.7142387763254
N_32:	 -2.49997000035364
V_32:	 2.14282775550507
M_32:	 -10.7140387787253
N_34:	 2.14282775550507
V_34:	 2.49997000036000
M_34:	 10.7140387787253
N_43:	 -2.14282775550507
V_43:	 -2.49997000036000
M_43:	 14.2856612248746


## Symbolic Solutions
Of course numerical answers are something that could be achieved faster and easier with a different program that has more functionality. The thing that makes this module useful is that we can solve structures symbolically using Sympy.

We will solve the following structure symbolically:

![Det Cant](img/simpleCant.png)

First define the beam characteristics with sympy symbols and constrain them to be positive to eliminate the trivial and negative solutions.

In [51]:
L, P, E, I= symbols('L P E I', positive=True)
A = 100000

Again define the structure and solve. However, we must require that the solution does not solve for the unknown beam characteristics using the exclude keyword argument.

In [52]:
struct2 = Structure()


struct2.add_node('1',(0,0))
struct2.add_node('2',(L,0), restraint='rolling_x')
struct2.add_node('3', (3*L, 0), restraint='fixed')

struct2.add_beam('1', (1,2), E=E, A=A, I=I)
struct2.add_beam('2', (2,3), E=E, A=A, I=I)

struct2.add_load('1', (0, -P, 0))

struct2.solve(exclude=[L, P, E, I])

Again we can get the displacements, reactions, and internal forces in the same way as before.

In [53]:
### displacements
for name, node in struct2.nodes.items():
    print(f'Delta_{name}x:\t', node.dx)
    print(f'Delta_{name}y:\t', node.dy)
    print(f'theta_{name}:\t', node.dt)

Delta_1x:	 0
Delta_1y:	 -5*L**3*P/(6*E*I)
theta_1:	 L**2*P/(E*I)
Delta_2x:	 0
Delta_2y:	 0
theta_2:	 L**2*P/(2*E*I)
Delta_3x:	 0
Delta_3y:	 0
theta_3:	 0


In [54]:
### Reactions
for name, node in struct2.nodes.items():
    print(f'F_{name}x:\t', node.fx)
    print(f'F_{name}y:\t', node.fy)
    print(f'M_{name}:\t', node.moment)

F_1x:	 0
F_1y:	 -P
M_1:	 0
F_2x:	 0
F_2y:	 7*P/4
M_2:	 0
F_3x:	 0
F_3y:	 -3*P/4
M_3:	 L*P/2


In [55]:
for name, beam in struct2.beams.items():
    beg = beam.nodeA.get_name()
    end = beam.nodeB.get_name()
    print(f'N_{beg}{end}:\t', beam.internal[0])
    print(f'V_{beg}{end}:\t', beam.internal[1])
    print(f'M_{beg}{end}:\t', beam.internal[2])
    print(f'N_{end}{beg}:\t', beam.internal[3])
    print(f'V_{end}{beg}:\t', beam.internal[4])
    print(f'M_{end}{beg}:\t', beam.internal[5])

N_12:	 0
V_12:	 -P
M_12:	 0
N_21:	 0
V_21:	 P
M_21:	 -L*P
N_23:	 0
V_23:	 3*P/4
M_23:	 L*P
N_32:	 0
V_32:	 -3*P/4
M_32:	 L*P/2
